# Get Button Press Stats

This script finds button presses statistics across all subjects. Copied from get_bpress_ALL_onsets_7-8. Tis the most recent version (i think)

In [30]:
import json
#Pandas for saving datasets
import pandas as pd
#matplotlib for rendering
import matplotlib.pyplot as plt
#numpy for handeling matrix operations
import numpy as np
#time, to, well... keep track of time
import time
#iPython display for making sure we can render the frames
from IPython import display
#seaborn for rendering
import seaborn
import math
import os
import statistics as stat
from scipy.stats import sem

# Find button presses / Counts 

In [2]:
## Reformats the button press array and returns the button press instances  ##
def org_bpress(arg):
    tem = len(arg.split())
    return [1] * tem
        

    
    

In [3]:
## Reformats the time array and returns the rounded (two decimals) time stamp  ##
## Timestamp represents the time the button was pressed, start of the run is t-zero ##
#returns an array of floats
import re
def org_tpress(arg):
    tem = arg.split()
    a=[]
    for i in tem:
        S = i
        pattern = re.compile(r'\-?\d+\.\d+')
        weights = list(map(float, re.findall(pattern, S)))
        a.append(round(float(weights[0]),2))
    #print(a)
    return(a)

    
    

In [4]:
## Returns the TR prior to the button press ## 
# arg is each time stamp: ([20.37, 45.95, 71.57, ...]) 
def tr_stam(arg):
    a = []
    for i in arg:
        a.append(int(round(i / 1.5,0)))
        #print(a)
    return(a)

In [5]:
## Returns a list of 209 TRs, 1 is a button press, 0 is not ## 
"""
all_TR_pres: 
 - iterate to num TRs (e.g. 210)
 - set the bpress TR (14,31,48) to 1, else the value is zero
 - returns: array of length num_trs with ones and zeros 
"""
def all_TR_pres(arg, num_trs):
    a = []
    for i in range(1,num_trs+1):
        if i in arg:
            a.append(1)
        else:
            a.append(0)
    return a
            

In [6]:
## Expand / Label TRs
"""
0 = SM
1 = SC
2 = OM
3 = OC
4 = Re
requires list of labels ouputed by psychopy (column 1 - MM_self_title.started, etc.)
returns label list (order is preserved) and TR labels
"""

def label_lists(label, num_tr):
    b = [[]]
    a = []
    for i in label:
        # substring label in psychopy output
        # if the first three characters == M_s, etc, then add correct indext to string
        if i[1:4] == "M_s":
            a.append("SM")
            b.append([0]*num_tr)
        elif i[1:4] == "C_s":
            a.append("SC")
            b.append([1]*num_tr)        
        elif i[1:4] == "M_o":
            a.append("OM")
            b.append([2]*num_tr)
        elif i[1:4] == "C_o":
            a.append("OC")
            b.append([3]*num_tr)     
        else:
            a.append("Re")
            b.append([4]*num_tr)     
    return a, b[1:]
 



In [7]:
def find_cond_index(sub_ses_labels):
    """
    For the array of ordered run names (i.e.'Re', 'SM',) find the two indexes per condition
    """ 
    lab_inx = []

    a = []
    b = []
    c = []
    d = []

    for i in enumerate(sub_ses_labels):
        if i[1] == "SM":
            # append the index according to where it appeared in the array
            a.append(i[0])
        if i[1] == "SC":
            b.append(i[0])
        if i[1] == "OM":
            c.append(i[0])
        if i[1] == "OC":
            d.append(i[0])

    # Create a dictionary where each key contains the appropriate indexes
    lab_indic = {
        'SM' : a,
        'SC' : b,
        'OM' : c,
        'OC' : d
    }
    return lab_indic 
    #np.vstack(lab_inx, ["SM", "SC", "OM", "OC"])

In [8]:
# Take in raw column data and return vector 0s and 1s, button press or no button press
"""
Input press data at e.g. sub.iloc[1,4]:
 - "['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']"
Input time data. This is the column immediately after :
 - '[20.37084077997133, 45.95158576697577, 71.56790153402835, ...] etc.
npress: reformat the above PRESS input data to integers
ts_stamp: reformat the above TIME input data into floats - time of runstart is t=0
tr_stam: take the reformatted data afrom ts_stamp ([20.37, 45.95, 71.57])
 - Divide timestamp, say 20.37, by 1.5. Result is 13.58. 
 - Round 13.58 up to 14
 - Between the 13th and 14th TR, the bpress occurs 
 - We round, so on the 14th TR the button press occurs 
 - Returns all of the TRs prior to the bpress: [14, 31, 48,..]
all_TR_pres: 
 - iterate to num TRs (e.g. 210)
 - set the bpress TR (14,31,48) to 1, else the value is zero
 - returns: array of length num_trs with ones and zeros 
"""
def run_press(pressN, timeN):
    n_press = org_bpress(pressN)
    ts_stamp = org_tpress(timeN)
    tr_stamp = tr_stam(ts_stamp)
    binar_210 = all_TR_pres(tr_stamp,num_trs)
    tr_210 = tr_stamp
    return binar_210, tr_210, ts_stamp

In [9]:
def bpress_arrays(dframe, sub_ses_labels, dtype):
    """
    Take in a column value (self or other) and count the number of presses for that particular run
    First get all self_btn info, add to an array. 
    Second, get all other info, add to an array
    All data is UNSORTED.
    args:
        cleaned subject dataframe
        dtype: tr_stamp = the tr when the button press occured
               ts_stamp = the time in seconds when the button press occured
               bin_arr = the binary array of when the button press occured (tr)
        
    return: 
        cond: unsorted array of labels (Self, then Other conds)
        Bpress: shape = 8 conditions by X TRs. 1s and 0s


    """
    if dtype == "tr_stamp":
        get_press = 1
    elif dtype == "ts_stamp":
        get_press= 2
    elif dtype == "bin_stamp":
        get_press= 0
    else:
        print("enter dtype arg correctly yo")
    
    df = []
    cond = []
    
    for col_ind, col_val in enumerate(dframe.columns):
        if col_val == "self_btn.keys":
            # For all Rows in the self column
            # if the row is of type string
            for row_inx, row_val in enumerate(dframe.iloc[0:,col_ind]):
                #if type(row_val) is type("a"):
                if row_val == "None":
                    print("hi")
                    df.append(np.array([]))
                    cond.append(sub_ses_labels[row_inx])
                elif row_val != "-1":
                    # Add to condition array
                    # There are 10 rows, and 10 runs. 
                    # I could pull from the third column to get the string name
                    # Instead i pull from the label order in sub_ses, which corresponds to the
                    # row_inx
                    cond.append(sub_ses_labels[row_inx])
                    """
                    run_press takes in the button presses, then the timing info (+1)
                    returns: a list of button presses, 1s and 0s for each run - [0]
                            - OR the TR when the button press occured - [1]
                    """
                    print("rowval:",row_inx, "colval:",col_ind, "dframe val:", dframe.iloc[row_inx,col_ind])
                    bpress_info = dframe.iloc[row_inx,col_ind]
                    time_info = dframe.iloc[row_inx,col_ind+1]
                    df.append(np.asarray(run_press(bpress_info, time_info)[get_press]).flatten())
                    tem = sum(run_press(dframe.iloc[row_inx,col_ind],dframe.iloc[row_inx,col_ind+1])[0])
                    print(sub_ses_labels[row_inx], "Sum bpress: ", tem)
        elif col_val == "other_j_btn1.keys" or col_val =="other_n_btn.keys":
            if sub == 'sub-000' and col_val =="other_n_btn.keys":
                print('hi')
                df.append(np.array([]))
                df.append(np.array([]))
                cond.append('OM')
                cond.append('OC')
            else:
                for row_inx, row_val in enumerate(dframe.iloc[0:,col_ind]):
                    if row_val == "None":
                        print("hi")
                        df.append(np.array([]))
                        cond.append(sub_ses_labels[row_inx])
                    elif row_val != "-1":
                        print("rowval:",row_inx, "colval:",col_ind, "dframe val:", dframe.iloc[row_inx,col_ind])
                        # Add to condition array
                        cond.append(sub_ses_labels[row_inx])
                        # append button press info
                        bpress_info = dframe.iloc[row_inx,col_ind]
                        time_info = dframe.iloc[row_inx,col_ind+1]
                        df.append(np.asarray(run_press(bpress_info, time_info)[get_press]).flatten())
                        tem = sum(run_press(dframe.iloc[row_inx,col_ind],dframe.iloc[row_inx,col_ind+1])[0])
                        print(sub_ses_labels[row_inx], "Sum bpress: ", tem)
    return np.asarray(df), cond



In [10]:
def sort_bpress_data(unsort_bpress,lab_indic_inc, lab_indic_cor):
    """
    1. find the correct index for SM, SC, etc.
    2. find the index of SM in the other dataframe
    3. set the empyt dataframe index of the correct SM to the data of the unsort
    4. Iterate
    """
    # create empty dframe for organized bpress data from shape of unsorted dframe
    sort_bpress = np.zeros(unsort_bpress.shape)
    print("Finished!")
    for i in range(0,2):
        sort_bpress[lab_indic_cor['SM'][i]] = unsort_bpress[lab_indic_inc['SM'][i]]
        sort_bpress[lab_indic_cor['SC'][i]] = unsort_bpress[lab_indic_inc['SC'][i]]
        sort_bpress[lab_indic_cor['OM'][i]] = unsort_bpress[lab_indic_inc['OM'][i]]
        sort_bpress[lab_indic_cor['OC'][i]] = unsort_bpress[lab_indic_inc['OC'][i]]
    return sort_bpress 
    
    
    
    
    
    

In [11]:
def sort_bpress_data_onset(unsort_bpress,lab_indic_inc, lab_indic_cor):
    """
    1. find the correct index for SM, SC, etc.
    2. find the index of SM in the other dataframe
    3. set the empyt dataframe index of the correct SM to the data of the unsort
    4. Iterate
    """
    # create empty dframe for organized bpress data from shape of unsorted dframe
    #sort_bpress = np.zeros(unsort_bpress.shape)
    onsets = [None] * len(unsort_bpress)
    
    for i in range(0,2):
        onsets[lab_indic_cor['SM'][i]] = unsort_bpress[lab_indic_inc['SM'][i]]
        onsets[lab_indic_cor['SC'][i]] = unsort_bpress[lab_indic_inc['SC'][i]]
        onsets[lab_indic_cor['OM'][i]] = unsort_bpress[lab_indic_inc['OM'][i]]
        onsets[lab_indic_cor['OC'][i]] = unsort_bpress[lab_indic_inc['OC'][i]]
    return onsets 
    
    
    
    
    
    

# Begin

In [12]:
behav_p = '/jukebox/graziano/coolCatIsaac/ATM/data/behavioral'
sav_work = "/jukebox/graziano/coolCatIsaac/ATM/data/work/results/"
sav_bpress = "/jukebox/graziano/coolCatIsaac/ATM/data/work/results/bpress_GLM/behav"
sub_list = ["sub-001","sub-003","sub-004","sub-005","sub-006","sub-007","sub-008","sub-009",
            "sub-011","sub-012","sub-013","sub-014"]
sub_list = ["sub-000","sub-001", "sub-002","sub-003","sub-004","sub-005","sub-006","sub-007","sub-008","sub-009",
            "sub-010","sub-011","sub-012","sub-013","sub-014","sub-015", "sub-016","sub-017", 
            "sub-018","sub-019", "sub-020","sub-021"]

sub_list = ["sub-000","sub-001", "sub-002","sub-003","sub-004","sub-005","sub-006","sub-007","sub-008","sub-009",
            "sub-010","sub-011","sub-012","sub-013","sub-014","sub-015", "sub-016","sub-017", 
            "sub-018","sub-019", "sub-020","sub-021","sub-022","sub-023","sub-024","sub-025","sub-026", "sub-027"]

#sub_list = ["test"]

# No sub-000 or sub-019 and an issue with sub-009! can you believe it
"""
Issues:
- sub-000: no button presses for other_n_monitor. values are 1.0 instead of "-1.0". Fucks errthang up **SOLVED
- sub-019: no button presses for other_n_monitor. Values is None, instead of "-1.0". Fucks errthang up **SOLVED
    * only one button press for other count! check effort... may be shit subject
- sub-009 only has one button press for SM **Not an issue for button press counts - just interval calculations
- sub-024 No button press for last self monitor condition! no eyetracking, maybe asleep! maybe should delete... ugh
"""
#sub_list = ['sub-015']
num_trs = 209

# SUM bpress per run# 

In [13]:
all_bpress = {}
for sub in sub_list:
    print("BEGIN: ", sub)
    # Load Behavioral
    behav = pd.read_csv(os.path.join(behav_p, '%s_behav_cleaned.csv') % (sub))
    if sub == 'sub-002': behav = behav.fillna("-1")
    label = behav.iloc[:,1]
    sub_ses_labels = label_lists(label,num_trs)
    # Get all button press data in array form, 1s and 0s. returns an array 8x209 
    # Other options: tr_stamp, bin_stamp, ts_stamp
    unsort_bpress, cond = bpress_arrays(behav, sub_ses_labels[0], "ts_stamp")
    # Find condition indexes for the unsorted + sorted data
    # Eliminate rest conditions from orignal run labeled data
    lab_indic_cor = find_cond_index(sub_ses_labels[0][1:-1])
    lab_indic_inc = find_cond_index(cond)
    # Returns 8x209 dataframe of 1s and 0s, sorted correctly by when the run occured
    sort_bpress = sort_bpress_data_onset(unsort_bpress,lab_indic_inc, lab_indic_cor)   
    sub_bpress_info = {}
    bpress_dat = {}
    summer = {}
    for lab in lab_indic_cor:
        ### SUM CouNT ###
        # Get the first, second index of the each condition #
        mn = np.hstack((len(sort_bpress[lab_indic_cor[lab][0]]),len(sort_bpress[lab_indic_cor[lab][1]])))
        summer[lab+"-1"] = len(sort_bpress[lab_indic_cor[lab][0]])
        summer[lab+"-2"] = len(sort_bpress[lab_indic_cor[lab][1]])
        print("the", lab,"count is", mn)

    all_bpress[sub] = summer
    print()
print('gametime baybee lets gooo!')

BEGIN:  sub-000
rowval: 1 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1']
SM Sum bpress:  6
rowval: 2 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  10
rowval: 7 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  10
rowval: 8 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SM Sum bpress:  13
rowval: 3 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OC Sum bpress:  12
rowval: 4 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OM Sum bpress:  14
hi
the SM count is [ 6 13]
the SC count is [10 10]
the OM count is [14  0]
the OC count is [12  0]

BEGIN:  sub-001
rowval: 1 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SM Sum bpress:  13
rowval: 3 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  8
rowval: 7 colval: 4 dfra

/jukebox/pkgs/PYGER/base/envs/0.11.0/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


rowval: 1 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SM Sum bpress:  17
rowval: 3 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  12
rowval: 6 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  11
rowval: 7 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SM Sum bpress:  24
rowval: 2 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OC Sum bpress:  16
rowval: 4 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OM Sum bpress:  29
rowval: 5 colval: 10 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OC Sum bpress:  13
rowval: 8 colval: 10

BEGIN:  sub-015
rowval: 1 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OC Sum bpress:  14
rowval: 4 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1']
OM Sum bpress:  7
rowval: 2 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  20
rowval: 3 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SM Sum bpress:  13
rowval: 6 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  20
rowval: 8 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SM Sum bpress:  14
rowval: 5 colval: 10 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OC Sum bpress:  13
rowval: 7 colval: 10 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1']
OM Sum bpress:  9
the SM count is [1

rowval: 1 colval: 6 dframe val: ['1', '1', '1', '1']
OM Sum bpress:  4
rowval: 3 colval: 6 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OC Sum bpress:  13
rowval: 2 colval: 9 dframe val: ['1', '1', '1', '1', '1', '1']
SC Sum bpress:  6
rowval: 4 colval: 9 dframe val: ['1', '1', '1', '1']
SM Sum bpress:  4
rowval: 7 colval: 9 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  10
hi
rowval: 5 colval: 12 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OC Sum bpress:  13
rowval: 6 colval: 12 dframe val: ['1', '1', '1', '1', '1']
OM Sum bpress:  5
the SM count is [4 0]
the SC count is [ 6 10]
the OM count is [4 5]
the OC count is [13 13]

BEGIN:  sub-025
rowval: 1 colval: 5 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OC Sum bpress:  13
rowval: 2 colval: 5 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1']
OM Sum bpress:  8
rowval: 3 colval: 9 dframe val: ['1', '1', 

In [14]:
dat= pd.DataFrame(all_bpress).T

In [15]:
dat

,SM-1,SM-2,SC-1,SC-2,OM-1,OM-2,OC-1,OC-2
sub-000,6,13,10,10,14,0,12,0
sub-001,13,14,8,9,12,13,13,12
sub-002,8,8,12,9,20,13,13,13
sub-003,7,11,12,10,7,12,13,13
sub-004,5,15,17,18,9,7,14,7
sub-005,9,8,7,9,18,15,13,14
sub-006,17,24,12,11,30,46,16,13
sub-007,15,17,16,13,8,5,11,12
sub-008,11,15,14,17,14,15,12,12
sub-009,1,1,9,14,1,1,14,12


In [35]:
#dat.to_csv('n28_bpress_counts.csv',index=False,sep='\t',mode='a')
!mv n28_bpress_counts.csv /jukebox/graziano/coolCatIsaac/ATM/data/behavioral

# BPRESS ONSETS # 

In [16]:
all_bpress = {}
for sub in sub_list:
    print("BEGIN: ", sub)
    # Load Behavioral
    behav = pd.read_csv(os.path.join(behav_p, '%s_behav_cleaned.csv') % (sub))
    if sub == 'sub-002': behav = behav.fillna("-1")
    label = behav.iloc[:,1]
    sub_ses_labels = label_lists(label,num_trs)
    # Get all button press data in array form, 1s and 0s. returns an array 8x209 
    # Other options: tr_stamp, bin_stamp, ts_stamp
    unsort_bpress, cond = bpress_arrays(behav, sub_ses_labels[0], "tr_stamp")
    # Find condition indexes for the unsorted + sorted data
    # Eliminate rest conditions from orignal run labeled data
    lab_indic_cor = find_cond_index(sub_ses_labels[0][1:-1])
    lab_indic_inc = find_cond_index(cond)
    # Returns 8x209 dataframe of 1s and 0s, sorted correctly by when the run occured
    sort_bpress = sort_bpress_data_onset(unsort_bpress,lab_indic_inc, lab_indic_cor)   
    sub_bpress_info = {}
    bpress_dat = {}
    summer = {}
    for lab in lab_indic_cor:
        temp_arr = []
        ### SUM CouNT ###
        # Get the first, second index of the each condition #
        #summer[lab+"-1"] = sort_bpress[lab_indic_cor[lab][0]]
        #summer[lab+"-2"] = sort_bpress[lab_indic_cor[lab][1]]
        temp_arr.append(sort_bpress[lab_indic_cor[lab][0]])
        temp_arr.append(sort_bpress[lab_indic_cor[lab][1]])
        summer[lab] = temp_arr
    all_bpress[sub] = summer
    print()
print('gametime baybee lets gooo!')

BEGIN:  sub-000
rowval: 1 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1']
SM Sum bpress:  6
rowval: 2 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  10
rowval: 7 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  10
rowval: 8 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SM Sum bpress:  13
rowval: 3 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OC Sum bpress:  12
rowval: 4 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OM Sum bpress:  14
hi

BEGIN:  sub-001
rowval: 1 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SM Sum bpress:  13
rowval: 3 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  8
rowval: 7 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  9
rowval: 8 colval: 4 dfra

/jukebox/pkgs/PYGER/base/envs/0.11.0/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


 2 colval: 7 dframe val: ['1']
SM Sum bpress:  1
rowval: 3 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  9
rowval: 5 colval: 7 dframe val: ['1']
SM Sum bpress:  1
rowval: 8 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  14
rowval: 6 colval: 10 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OC Sum bpress:  12
rowval: 7 colval: 10 dframe val: ['1']
OM Sum bpress:  1

BEGIN:  sub-010
rowval: 1 colval: 4 dframe val: ['1', '1', '1', '1', '1']
OC Sum bpress:  5
rowval: 4 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OM Sum bpress:  20
rowval: 2 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  12
rowval: 3 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SM Sum bpress:  13
rowval: 5 colval: 7 dframe val: [

OM Sum bpress:  5

BEGIN:  sub-025
rowval: 1 colval: 5 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OC Sum bpress:  13
rowval: 2 colval: 5 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1']
OM Sum bpress:  8
rowval: 3 colval: 9 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  9
rowval: 4 colval: 9 dframe val: ['1', '1', '1', '1', '1']
SM Sum bpress:  5
rowval: 7 colval: 9 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  8
rowval: 8 colval: 9 dframe val: ['1', '1', '1', '1', '1', '1']
SM Sum bpress:  6
rowval: 5 colval: 12 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1']
OM Sum bpress:  8
rowval: 6 colval: 12 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OC Sum bpress:  12

BEGIN:  sub-026
rowval: 1 colval: 5 dframe val: ['1', '1', '1']
SM Sum bpress:  3
rowval: 3 colval: 5 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  8
rowval: 7 colval: 5 dframe val: ['

In [26]:
all_bpress['sub-024']

{'SM': [array([ 78, 115, 148, 164]), array([], dtype=float64)],
 'SC': [array([ 23,  45,  64,  80, 108, 170]),
  array([ 20,  37,  55,  76,  95, 113, 133, 151, 171, 189])],
 'OM': [array([ 15,  29,  57, 129]), array([ 57,  79, 100, 133, 167])],
 'OC': [array([ 12,  27,  42,  55,  70,  84,  95, 113, 127, 141, 154, 170, 189]),
  array([ 16,  31,  47,  62,  78,  94, 111, 126, 141, 155, 171, 185, 203])]}

In [27]:
#np.save(os.path.join(sav_bpress, "n28_4_conds_tr_stamp.npy"), all_bpress)

In [113]:
!pwd

/mnt/cup/labs/graziano/coolCatIsaac/ATM/code/analysis/bpress


In [20]:
sav_bpress

'/jukebox/graziano/coolCatIsaac/ATM/data/work/results/bpress_GLM/behav'

In [28]:
sub_list = ['sub-001']

# nick and josh bpress separately # 

In [18]:
def bpress_arrays(dframe, sub_ses_labels, dtype):
    """
    Take in a column value (self or other) and count the number of presses for that particular run
    First get all self_btn info, add to an array. 
    Second, get all other info, add to an array
    All data is UNSORTED.
    args:
        cleaned subject dataframe
        dtype: tr_stamp = the tr when the button press occured
               ts_stamp = the time in seconds when the button press occured
               bin_arr = the binary array of when the button press occured (tr)
        
    return: 
        cond: unsorted array of labels (Self, then Other conds)
        Bpress: shape = 8 conditions by X TRs. 1s and 0s


    """
    if dtype == "tr_stamp":
        get_press = 1
    elif dtype == "ts_stamp":
        get_press= 2
    elif dtype == "bin_stamp":
        get_press= 0
    else:
        print("enter dtype arg correctly yo")
    
    df = []
    cond = []
    
    for col_ind, col_val in enumerate(dframe.columns):
        if col_val == "self_btn.keys":
            # For all Rows in the self column
            # if the row is of type string
            for row_inx, row_val in enumerate(dframe.iloc[0:,col_ind]):
                #if type(row_val) is type("a"):
                if row_val != "-1":
                    # Add to condition array
                    # There are 10 rows, and 10 runs. 
                    # I could pull from the third column to get the string name
                    # Instead i pull from the label order in sub_ses, which corresponds to the
                    # row_inx
                    cond.append(sub_ses_labels[row_inx])
                    """
                    run_press takes in the button presses, then the timing info (+1)
                    returns: a list of button presses, 1s and 0s for each run - [0]
                            - OR the TR when the button press occured - [1]
                    """
                    print("rowval:",row_inx, "colval:",col_ind, "dframe val:", dframe.iloc[row_inx,col_ind])
                    bpress_info = dframe.iloc[row_inx,col_ind]
                    time_info = dframe.iloc[row_inx,col_ind+1]
                    df.append(np.asarray(run_press(bpress_info, time_info)[get_press]).flatten())
                    tem = sum(run_press(dframe.iloc[row_inx,col_ind],dframe.iloc[row_inx,col_ind+1])[0])
                    print(sub_ses_labels[row_inx], "Sum bpress: ", tem)
        elif col_val == "other_j_btn1.keys":
            if sub == 'sub-000' and col_val =="other_n_btn.keys":
                print('hi')
                df.append(np.array([]))
                df.append(np.array([]))
                cond.append('OM-N')
                cond.append('OC-N')
            else:
                for row_inx, row_val in enumerate(dframe.iloc[0:,col_ind]):
                    if row_val == "None":
                        print("hi")
                        df.append(np.array([]))
                        cond.append(sub_ses_labels[row_inx])
                    elif row_val != "-1":
                        print("rowval:",row_inx, "colval:",col_ind, "dframe val:", dframe.iloc[row_inx,col_ind])
                        # Add to condition array
                        cond.append(sub_ses_labels[row_inx])
                        # append button press info
                        bpress_info = dframe.iloc[row_inx,col_ind]
                        time_info = dframe.iloc[row_inx,col_ind+1]
                        df.append(np.asarray(run_press(bpress_info, time_info)[get_press]).flatten())
                        tem = sum(run_press(dframe.iloc[row_inx,col_ind],dframe.iloc[row_inx,col_ind+1])[0])
                        print(sub_ses_labels[row_inx], "Sum bpress: ", tem)
        elif col_val =="other_n_btn.keys":
            if sub == 'sub-000' and col_val =="other_n_btn.keys":
                print('hi')
                df.append(np.array([]))
                df.append(np.array([]))
                cond.append('OM')
                cond.append('OC')
            else:
                for row_inx, row_val in enumerate(dframe.iloc[0:,col_ind]):
                    if row_val == "None":
                        print("hi")
                        df.append(np.array([]))
                        cond.append(sub_ses_labels[row_inx])
                    elif row_val != "-1":
                        print("rowval:",row_inx, "colval:",col_ind, "dframe val:", dframe.iloc[row_inx,col_ind])
                        # Add to condition array
                        cond.append(sub_ses_labels[row_inx])
                        print(sub_ses_labels[row_inx], "nick", row_inx)
                        # append button press info
                        bpress_info = dframe.iloc[row_inx,col_ind]
                        time_info = dframe.iloc[row_inx,col_ind+1]
                        df.append(np.asarray(run_press(bpress_info, time_info)[get_press]).flatten())
                        tem = sum(run_press(dframe.iloc[row_inx,col_ind],dframe.iloc[row_inx,col_ind+1])[0])
                        print(sub_ses_labels[row_inx], "Sum bpress: ", tem)
    return np.asarray(df), cond



In [15]:
all_bpress = {}
for sub in sub_list:
    print("BEGIN: ", sub)
    # Load Behavioral
    behav = pd.read_csv(os.path.join(behav_p, '%s_behav_cleaned.csv') % (sub))
    if sub == 'sub-002': behav = behav.fillna("-1")
    label = behav.iloc[:,1]
    sub_ses_labels = label_lists(label,num_trs)
    # Get all button press data in array form, 1s and 0s. returns an array 8x209 
    # Other options: tr_stamp, bin_stamp, ts_stamp
    unsort_bpress, cond = bpress_arrays(behav, sub_ses_labels[0], "tr_stamp")
    # Find condition indexes for the unsorted + sorted data
    # Eliminate rest conditions from orignal run labeled data
    lab_indic_cor = find_cond_index(sub_ses_labels[0][1:-1])
    lab_indic_inc = find_cond_index(cond)
    # Returns 8x209 dataframe of 1s and 0s, sorted correctly by when the run occured
    sort_bpress = sort_bpress_data_onset(unsort_bpress,lab_indic_inc, lab_indic_cor)   
    sub_bpress_info = {}
    bpress_dat = {}
    summer = {}
    for lab in lab_indic_cor:
        temp_arr = []
        ### SUM CounT ###
        # Get the first, second index of the each condition #
        #summer[lab+"-1"] = sort_bpress[lab_indic_cor[lab][0]]
        #summer[lab+"-2"] = sort_bpress[lab_indic_cor[lab][1]]
        temp_arr.append(sort_bpress[lab_indic_cor[lab][0]])
        temp_arr.append(sort_bpress[lab_indic_cor[lab][1]])
        summer[lab] = temp_arr
    all_bpress[sub] = summer
    print()
print('gametime baybee lets gooo!')

BEGIN:  sub-000
rowval: 1 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1']
SM Sum bpress:  6
rowval: 2 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  10
rowval: 7 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  10
rowval: 8 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SM Sum bpress:  13
rowval: 3 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OC Sum bpress:  12
rowval: 4 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OM Sum bpress:  14
hi

BEGIN:  sub-001
rowval: 1 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SM Sum bpress:  13
rowval: 3 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  8
rowval: 7 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  9
rowval: 8 colval: 4 dfra

/jukebox/pkgs/PYGER/base/envs/0.11.0/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)



BEGIN:  sub-008
rowval: 1 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OC nick 1
OC Sum bpress:  12
rowval: 3 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OM nick 3
OM Sum bpress:  14
rowval: 2 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  14
rowval: 4 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SM Sum bpress:  11
rowval: 6 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SM Sum bpress:  15
rowval: 7 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  17
rowval: 5 colval: 10 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OC Sum bpress:  12
rowval: 8 colval: 10 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OM Sum bpress:

rowval: 1 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1']
OM Sum bpress:  8
rowval: 2 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OC Sum bpress:  13
rowval: 3 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SM Sum bpress:  18
rowval: 4 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  20
rowval: 6 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  18
rowval: 7 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SM Sum bpress:  26
rowval: 5 colval: 10 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OM nick 5
OM Sum bpress:  17
rowval: 8 colval: 10 dfram

IndexError: list index out of range

In [19]:
all_bpress['sub-000']

{'SM': [array([ 63,  77,  84, 103, 148, 179]),
  array([ 19,  40,  50,  69,  84, 103, 112, 127, 139, 150, 168, 184, 199])],
 'SC': [array([ 16,  32,  47,  65,  83, 101, 122, 143, 168, 188]),
  array([ 14,  28,  42,  59,  79, 100, 125, 144, 170, 200])],
 'OM': [array([ 17,  30,  48,  62,  74,  96, 102, 136, 156, 167, 180, 184, 191,
         198]),
  array([], dtype=float64)],
 'OC': [array([ 16,  32,  48,  63,  78,  94, 111, 126, 140, 155, 171, 192]),
  array([], dtype=float64)]}

# Get all bpress stats

In [14]:
## matrix of statistics at the subject level
all_bpress = {}


## create group level data
group_bpress_stats = {}

for sub in sub_list:
    print("BEGIN: ", sub)
    # Load Behavioral
    behav = pd.read_csv(os.path.join(behav_p, '%s_behav_cleaned.csv') % (sub))
    if sub == 'sub-002': behav = behav.fillna("-1")
    label = behav.iloc[:,1]
    sub_ses_labels = label_lists(label,num_trs)
    # Get all button press data in array form, 1s and 0s. returns an array 8x209 
    # Other options: tr_stamp, bin_stamp, ts_stamp
    unsort_bpress, cond = bpress_arrays(behav, sub_ses_labels[0], "ts_stamp")
    # Find condition indexes for the unsorted + sorted data
    # Eliminate rest conditions from orignal run labeled data
    lab_indic_cor = find_cond_index(sub_ses_labels[0][1:-1])
    lab_indic_inc = find_cond_index(cond)
    
    # Returns 8x209 dataframe of 1s and 0s, sorted correctly by when the run occured
    sort_bpress = sort_bpress_data_onset(unsort_bpress,lab_indic_inc, lab_indic_cor) 
    # create empty dic for subject 
    sub_bpress_info = {}
    # separate each condition by run
    summer = {}
    for lab in lab_indic_cor:
        ### SUM CouNT ###
        # Get the first, second index of the each condition #
        mn = np.hstack((len(sort_bpress[lab_indic_cor[lab][0]]),len(sort_bpress[lab_indic_cor[lab][1]])))
        
        # create subject specifc data
        summer[lab+"-1"] = len(sort_bpress[lab_indic_cor[lab][0]])
        summer[lab+"-2"] = len(sort_bpress[lab_indic_cor[lab][1]])
        print("the", lab,"count is", mn)
        
        ### Interval Info ###
        # Create empty array with a length of the number of button presses
        # minus one cuz we subtract pairs in the array from one another
        # For each run in the condition
        print(lab, "differences are:")
        # merge both runs into one array
        cond_diff = []
        # mean / vars dict
        calcs = {}
        for run in range(2):
            run_bpress_arr = sort_bpress[lab_indic_cor[lab][run]]
            # Insert zero for the start of the task
            run_bpress_arr = np.insert(run_bpress_arr,0,0) 
            difs = np.zeros(len(run_bpress_arr)-1)
            for idx in range(1,len(run_bpress_arr)):
                # set the current index of the empty array (diffs) to the subtracted value
                # of the current bpress idx minus the bpress -1 index
                difs[idx-1] = run_bpress_arr[idx] - run_bpress_arr[idx-1]
            print(difs)
            cond_diff.append(difs)
        # now calculate means and differences 
        # first merge two runs
        merg = np.concatenate(cond_diff)
        # run calcs on individual subject and save
        calcs = {'mean_int': np.mean(merg), 'sd_int': stat.stdev(merg), 'av_press': np.mean(mn)}
        # create sub dic
        sub_bpress_info[lab] = calcs
        
        ## add to group data 
        stats = {'Press' : mn, 'Interval' : merg}
        if lab not in group_bpress_stats:
            group_bpress_stats[lab] = stats
        else:
            group_bpress_stats[lab]['Press'] = np.hstack((group_bpress_stats[lab]['Press'], stats['Press']))
            group_bpress_stats[lab]['Interval'] = np.hstack((group_bpress_stats[lab]['Interval'], stats['Interval']))
        
    all_bpress[sub] = sub_bpress_info
    print()
print('gametime baybee lets gooo!')

BEGIN:  sub-000
rowval: 1 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1']
SM Sum bpress:  6
rowval: 2 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  10
rowval: 7 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  10
rowval: 8 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SM Sum bpress:  13
rowval: 3 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OC Sum bpress:  12
rowval: 4 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OM Sum bpress:  14
hi
the SM count is [ 6 13]
SM differences are:
[94.6  21.    9.95 28.47 67.81 47.21]
[27.81 31.52 15.66 28.15 23.61 27.54 13.62 22.48 17.76 17.36 26.43 24.4
 22.77]
the SC count is [10 10]
SC differences are:
[24.51 22.9  23.14 26.25 27.71 27.6  30.39 31.81 38.3  28.84]
[21.66 20.4  21.61 25.26 29.34 31.85 36.97 29.21 39.01 44.4 ]
the OM count is

BEGIN:  sub-008
rowval: 1 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OC Sum bpress:  12
rowval: 3 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OM Sum bpress:  14
rowval: 2 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  14
rowval: 4 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SM Sum bpress:  11
rowval: 6 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SM Sum bpress:  15
rowval: 7 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  17
rowval: 5 colval: 10 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OC Sum bpress:  12
rowval: 8 colval: 10 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OM Sum bpress:  15
the SM count is 

the OM count is [7 9]
OM differences are:
[40.08 34.25  8.6  72.72 33.1  32.43 17.35]
[110.37  16.61  24.99  82.76  21.6   13.65   6.62  10.33   9.75]
the OC count is [14 13]
OC differences are:
[17.92 21.28 23.28 19.47 22.4  21.3  21.05 28.97 17.78 21.91 19.61 30.31
 21.78 17.97]
[23.85 22.98 23.8  22.4  23.25 24.29 24.74 22.51 22.41 21.25 22.65 22.98
 27.84]

BEGIN:  sub-016
rowval: 1 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OC Sum bpress:  12
rowval: 4 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1']
OM Sum bpress:  8
rowval: 2 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SM Sum bpress:  11
rowval: 3 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  21
rowval: 5 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SM Sum bpress:  14
rowval: 6 colval: 7 dframe val:

SC Sum bpress:  9
rowval: 4 colval: 9 dframe val: ['1', '1', '1', '1', '1']
SM Sum bpress:  5
rowval: 7 colval: 9 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  8
rowval: 8 colval: 9 dframe val: ['1', '1', '1', '1', '1', '1']
SM Sum bpress:  6
rowval: 5 colval: 12 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1']
OM Sum bpress:  8
rowval: 6 colval: 12 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OC Sum bpress:  12
the SM count is [5 6]
SM differences are:
[70.86 80.32 61.8  57.81 25.86]
[77.77 43.46 35.39 27.05 59.38 51.72]
the SC count is [9 8]
SC differences are:
[30.56 32.56 32.45 30.77 28.38 26.98 29.27 29.31 34.17]
[32.35 48.31 31.31 44.88 30.62 31.7  29.8  30.1 ]
the OM count is [8 8]
OM differences are:
[40.18 28.16 16.18 58.66 39.07 44.73 37.45 24.45]
[51.58 39.07 21.35 39.73 53.97 25.22 38.49 26.52]
the OC count is [13 12]
OC differences are:
[22.3  23.05 19.58 21.05 22.4  21.6  21.62 30.4  18.03 21.63 21.99 26.63
 20.18]
[2

In [62]:

dat_all = {}
for cond in group_bpress_stats:
    stat_dic = {}
    for my_stat in group_bpress_stats[cond]:
        dat_stat =  group_bpress_stats[cond][my_stat]
        stat_dic[f"{my_stat} Mean"] = np.round(np.mean(dat_stat), 2) 
        stat_dic[f"{my_stat} StD Error"] = np.round(sem(dat_stat), 4)
                    
        print(stat_dic)
    dat_all[cond] = stat_dic
        
        

{'Press Mean': 10.86, 'Press StD Error': 0.8088}
{'Press Mean': 10.86, 'Press StD Error': 0.8088, 'Interval Mean': 25.68, 'Interval StD Error': 0.9077}
{'Press Mean': 12.05, 'Press StD Error': 0.5305}
{'Press Mean': 12.05, 'Press StD Error': 0.5305, 'Interval Mean': 24.18, 'Interval StD Error': 0.4481}
{'Press Mean': 11.16, 'Press StD Error': 1.0032}
{'Press Mean': 11.16, 'Press StD Error': 1.0032, 'Interval Mean': 23.86, 'Interval StD Error': 0.8776}
{'Press Mean': 11.75, 'Press StD Error': 0.3933}
{'Press Mean': 11.75, 'Press StD Error': 0.3933, 'Interval Mean': 24.26, 'Interval StD Error': 0.4401}


In [63]:
df = pd.DataFrame(dat_all).T
df

,Press Mean,Press StD Error,Interval Mean,Interval StD Error
SM,10.86,0.8088,25.68,0.9077
SC,12.05,0.5305,24.18,0.4481
OM,11.16,1.0032,23.86,0.8776
OC,11.75,0.3933,24.26,0.4401


In [64]:
temp = pd.DataFrame(dat_all)
row_labels = {"Press Mean" : "Button Press Mean",
             "Press StD Error" : "Button Press StD Error ",
              "Interval Mean" : "Button Press Interval Mean" , 
              "Interval Std Error" : "Button Press Interval StD Error"
             ""}
column_labels = {"SM" : "Self Monitor",
                "SC": "Self Count",
                "OM" : "Other Monitor", 
                "OC" : "Other Count"}
temp.rename(index = row_labels, columns = column_labels, inplace=True)
df = temp.T
df

,Button Press Mean,Button Press StD Error,Button Press Interval Mean,Interval StD Error
Self Monitor,10.86,0.8088,25.68,0.9077
Self Count,12.05,0.5305,24.18,0.4481
Other Monitor,11.16,1.0032,23.86,0.8776
Other Count,11.75,0.3933,24.26,0.4401


In [60]:
def center_align(val):
    return 'text-align: center'

In [61]:
styled_df = df.style.applymap(center_align)
#styled_df.round(4)

AttributeError: 'Styler' object has no attribute 'round'

In [57]:
styled_df

,Button Press Mean,Button StD Error,Button Press Interval Mean,Interval StD Error
Self Monitor,10.860000,0.808800,25.680000,0.907700
Self Count,12.050000,0.530500,24.180000,0.448100
Other Monitor,11.160000,1.003200,23.860000,0.877600
Other Count,11.750000,0.393300,24.260000,0.440100


In [ ]:
pd.to_csv(sav_work + "bpress_statistics_publication.csv", df)


In [49]:
sav_work

'/jukebox/graziano/coolCatIsaac/ATM/data/work/results/'

In [48]:
group_bpress_stats['SM']

{'Press': array([ 6, 13, 13, 14,  8,  8,  7, 11,  5, 15,  9,  8, 17, 24, 15, 17, 11,
        15,  1,  1, 13, 16, 17, 10, 13, 10, 12, 11,  1,  8, 13, 14, 11, 14,
         8, 17,  4,  4,  4,  8, 10, 16, 18, 26,  9, 22,  7,  7,  4,  0,  5,
         6,  3,  6, 21, 22]),
 'Interval': array([ 94.6 ,  21.  ,   9.95,  28.47,  67.81,  47.21,  27.81,  31.52,
         15.66,  28.15,  23.61,  27.54,  13.62,  22.48,  17.76,  17.36,
         26.43,  24.4 ,  22.77,   7.8 ,  23.05,  22.43,  13.73,  49.29,
         28.4 ,  17.87,  16.17,  21.89,  29.75,  17.41,  15.19,  27.38,
          2.58,  10.4 ,   8.67,   5.85,  20.2 ,  24.68,   3.67,  30.99,
         38.26,  22.05,  25.98,  34.95,  32.91,  12.  ,   8.53,  44.98,
         52.59,  30.56,  48.87,  24.05,  14.6 ,  77.7 ,  60.5 ,  20.95,
         18.24,  75.43,  29.65,  30.11,  51.52,  19.9 ,  51.47,  35.19,
         38.41,  19.45,  42.53,  44.49,  55.03,  28.29,  19.18,  25.85,
         30.95,  13.3 ,  44.98,  47.92,  17.15,  15.97,  33.86,  22.42,
 

In [24]:
bpress_dat

{}

In [ ]:
## previoius

In [17]:
all_bpress = {}
for sub in sub_list:
    print("BEGIN: ", sub)
    # Load Behavioral
    behav = pd.read_csv(os.path.join(behav_p, '%s_behav_cleaned.csv') % (sub))
    label = behav.iloc[:,1]
    sub_ses_labels = label_lists(label,num_trs)
    # Get all button press data in array form, 1s and 0s. returns an array 8x209 
    # Other options: tr_stamp, bin_stamp, ts_stamp
    unsort_bpress, cond = bpress_arrays(behav, sub_ses_labels[0], "ts_stamp")
    # Find condition indexes for the unsorted + sorted data
    # Eliminate rest conditions from orignal run labeled data
    lab_indic_cor = find_cond_index(sub_ses_labels[0][1:-1])
    lab_indic_inc = find_cond_index(cond)
    # Returns 8x209 dataframe of 1s and 0s, sorted correctly by when the run occured
    sort_bpress = sort_bpress_data_onset(unsort_bpress,lab_indic_inc, lab_indic_cor)   
    sub_bpress_info = {}
    bpress_dat = {}
    for lab in lab_indic_cor:
        ### SUM CounT ###
        # Get the first, second index of the each condition #
        mn = np.hstack((len(sort_bpress[lab_indic_cor[lab][0]]),len(sort_bpress[lab_indic_cor[lab][1]])))
        #summer[lab] = np.mean(mn)
        print("the", lab,"count is", mn)
        #for idx in lab_indic_inc[lab]:
        ### Interval Info ###
        # Create empty array with a length of the number of button presses
        # minus one cuz we subtract pairs in the array from one another
        # For each run in the condition
        print(lab, "differences are:")
        # merge both runs into one array
        cond_diff = []
        # mean / vars dict
        calcs = {}
        for run in range(2):
            run_bpress_arr = sort_bpress[lab_indic_cor[lab][run]]
            # Insert zero for the start of the task
            run_bpress_arr = np.insert(run_bpress_arr,0,0) 
            difs = np.zeros(len(run_bpress_arr)-1)
            for idx in range(1,len(run_bpress_arr)):
                # set the current index of the empty array (diffs) to the subtracted value
                # of the current bpress idx minus the bpress -1 index
                difs[idx-1] = run_bpress_arr[idx] - run_bpress_arr[idx-1]
            print(difs)
            cond_diff.append(difs)
        # now calculate means and differences 
        # first merge two runs
        merg = np.concatenate(cond_diff)
        calcs = {'mean_int': np.mean(merg), 'sd_int': stat.stdev(merg), 'av_press': np.mean(mn)}
        sub_bpress_info[lab] = calcs
    all_bpress[sub] = sub_bpress_info
    print()
print('gametime baybee lets gooo!')

BEGIN:  sub-000
rowval: 1 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1']
SM Sum bpress:  6
rowval: 2 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  10
rowval: 7 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  10
rowval: 8 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SM Sum bpress:  13
rowval: 3 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OC Sum bpress:  12
rowval: 4 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OM Sum bpress:  14
hi
the SM count is [ 6 13]
SM differences are:
[94.6  21.    9.95 28.47 67.81 47.21]
[27.81 31.52 15.66 28.15 23.61 27.54 13.62 22.48 17.76 17.36 26.43 24.4
 22.77]
the SC count is [10 10]
SC differences are:
[24.51 22.9  23.14 26.25 27.71 27.6  30.39 31.81 38.3  28.84]
[21.66 20.4  21.61 25.26 29.34 31.85 36.97 29.21 39.01 44.4 ]
the OM count is

AttributeError: 'float' object has no attribute 'split'

In [15]:
merg

array([18.3 , 21.8 , 21.91, 20.8 , 22.14, 20.68, 22.6 , 21.78, 23.54,
       29.48, 21.16, 29.2 , 31.47, 23.57, 23.16, 23.9 , 23.4 , 23.05,
       23.57, 25.03, 23.07, 28.1 , 24.58, 23.27, 28.13])

In [30]:
lab_indic_inc

{'SM': [2, 5], 'SC': [3, 4], 'OM': [0, 6], 'OC': [1, 7]}

In [31]:
lab_indic_cor

{'SM': [2, 6], 'SC': [3, 5], 'OM': [0, 4], 'OC': [1, 7]}

In [21]:
sort_bpress

[array([  7.8 ,  30.85,  53.28,  67.01, 116.3 , 144.7 , 162.57, 178.74,
        200.63, 230.38, 247.79, 262.98, 290.36]),
 array([ 13.35,  39.75,  75.18,  82.57,  85.  , 112.55, 155.96, 188.75,
        200.98, 221.06, 222.58, 238.51]),
 array([ 42.98,  85.26, 121.66, 161.35, 196.7 , 228.55, 254.42, 285.95]),
 array([ 18.3 ,  40.1 ,  62.01,  82.81, 104.95, 125.63, 148.23, 170.01,
        193.55, 223.03, 244.19, 273.39, 304.86]),
 array([ 34.31,  80.84, 101.98, 107.63, 112.46, 153.61, 156.56, 203.79,
        234.54, 269.91, 275.49, 287.09, 294.51]),
 array([ 23.57,  46.73,  70.63,  94.03, 117.08, 140.65, 165.68, 188.75,
        216.85, 241.43, 264.7 , 292.83]),
 array([ 40.25,  74.61, 102.55, 132.45, 164.88, 197.5 , 226.1 , 254.84,
        288.92]),
 array([  2.58,  12.98,  21.65,  27.5 ,  47.7 ,  72.38,  76.05, 107.04,
        145.3 , 167.35, 193.33, 228.28, 261.19, 273.19])]

In [32]:
df = pd.DataFrame(all_bpress)
df = df.T

In [33]:
chek = pd.DataFrame.from_dict({(i,j): df[i][j] 
                           for i in df.keys() 
                           for j in df[i].keys()})


In [16]:

###### OG ####
chek['SM'].T

,mean_int,sd_int,av_press
sub-001,23.713333,7.213056,13.0
sub-003,27.786667,10.705369,10.0
sub-004,24.395500,16.217508,11.0
sub-005,22.234545,9.439308,12.0
sub-006,12.533111,8.911257,23.5
sub-007,22.349167,8.096207,13.0
sub-008,25.963333,10.028908,11.5
sub-011,32.323750,19.777697,9.0
sub-012,20.680833,8.702605,13.0
sub-013,23.621304,4.572161,12.5


In [34]:
chek['SM'].T

,mean_int,sd_int,av_press
sub-001,20.872222,11.222561,13.5
sub-003,32.580000,13.151676,9.0
sub-004,29.767000,20.285543,10.0
sub-005,34.494706,14.326242,8.5
sub-006,14.751220,9.670633,20.5
sub-007,18.650938,6.687291,16.0
sub-008,23.031538,10.604360,13.0
sub-011,22.435185,18.755129,13.5
sub-012,23.760000,15.538411,11.5
sub-013,25.627391,13.851810,11.5


In [35]:
sm = chek['SM'].T

In [36]:
# mean bpress
np.mean(sm.values[:,2])

12.294117647058824

In [37]:
# std bpress
stat.stdev(sm.values[:,2])

4.623509298713924

In [38]:
# mean interval
np.mean(sm.values[:,0])

26.96618247729081

In [39]:
# sd interval
np.median(sm.values[:,1])

13.851809930110973

In [40]:
# sd interval
np.mean(sm.values[:,1])

14.446007607890174

In [80]:
dict0 = {'sum':15, 'mean_dir': 10.2, 'var':5}

In [84]:
dict1 = {'SM':np.asarray([15,10.3, 5]), 'OM':np.asarray([15,10.3, 5])}
dict1 = {'SM':dict0, 'OM':dict0}
dict2 = {'sub-001': dict1, 'sub-002': dict1,'sub-003': dict1,'sub-004': dict1}

In [85]:
df = pd.DataFrame(dict2)
df = df.T

In [99]:
chek = pd.DataFrame.from_dict({(i,j): df[i][j] 
                           for i in df.keys() 
                           for j in df[i].keys()})


In [92]:
chek['sum']

SM  sub-001    15
    sub-002    15
    sub-003    15
    sub-004    15
OM  sub-001    15
    sub-002    15
    sub-003    15
    sub-004    15
Name: sum, dtype: int64

In [102]:
chek['SM'].T

,sum,mean_dir,var
sub-001,15.0,10.2,5.0
sub-002,15.0,10.2,5.0
sub-003,15.0,10.2,5.0
sub-004,15.0,10.2,5.0


In [98]:
chek.set_names(['cond', 'sub'], inplace = True)

AttributeError: 'DataFrame' object has no attribute 'set_names'

In [94]:
chek.index

MultiIndex([('SM', 'sub-001'),
            ('SM', 'sub-002'),
            ('SM', 'sub-003'),
            ('SM', 'sub-004'),
            ('OM', 'sub-001'),
            ('OM', 'sub-002'),
            ('OM', 'sub-003'),
            ('OM', 'sub-004')],
           )

In [96]:
chek[chek.index['SM']]

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [52]:
df.T

,SM,OM
sub-001,"[15.0, 10.3, 5.0]","[15.0, 10.3, 5.0]"
sub-002,"[15.0, 10.3, 5.0]","[15.0, 10.3, 5.0]"
sub-003,"[15.0, 10.3, 5.0]","[15.0, 10.3, 5.0]"
sub-004,"[15.0, 10.3, 5.0]","[15.0, 10.3, 5.0]"


In [56]:
ew = np.asarray(df.T)

In [59]:
ew

array([[array([15. , 10.3,  5. ]), array([15. , 10.3,  5. ])],
       [array([15. , 10.3,  5. ]), array([15. , 10.3,  5. ])],
       [array([15. , 10.3,  5. ]), array([15. , 10.3,  5. ])],
       [array([15. , 10.3,  5. ]), array([15. , 10.3,  5. ])]],
      dtype=object)

In [60]:
ew.flatten()

array([array([15. , 10.3,  5. ]), array([15. , 10.3,  5. ]),
       array([15. , 10.3,  5. ]), array([15. , 10.3,  5. ]),
       array([15. , 10.3,  5. ]), array([15. , 10.3,  5. ]),
       array([15. , 10.3,  5. ]), array([15. , 10.3,  5. ])], dtype=object)

In [61]:
# Get SM
ew = ew[:,1]

In [63]:
ew.flatten()

array([array([15. , 10.3,  5. ]), array([15. , 10.3,  5. ]),
       array([15. , 10.3,  5. ]), array([15. , 10.3,  5. ])], dtype=object)

In [64]:
ew

array([array([15. , 10.3,  5. ]), array([15. , 10.3,  5. ]),
       array([15. , 10.3,  5. ]), array([15. , 10.3,  5. ])], dtype=object)

In [62]:
sort_bpress[7]

array([ 21.06,  43.05,  62.29,  99.26, 131.41, 151.43, 169.94, 188.29,
       203.31, 219.21, 241.89, 265.86, 283.69])

In [56]:
sort_bpress[0]

array([ 34.81,  43.16, 111.25, 128.11, 162.16, 237.76, 280.66, 294.59])

In [57]:
print(difs)

[ 8.35 68.09 16.86 34.05 75.6  42.9  13.93]


In [50]:
sort_bpress[0][0]

34.81

In [51]:
sort_bpress[0][1] - sort_bpress[0][0]

8.349999999999994

In [25]:
len(sort_bpress[5])

14

In [18]:
## list 
#Inner loop - across runs #
count_bpress = []
#outisde loop - across subjects
# COUNT BPRESS
sub_bpress = []
# CONDITION LABEL
cond_arr = []

In [60]:
cond_arr.append(lab_indic_inc)

In [61]:
cond_arr

[{'SM': [0, 3], 'SC': [1, 2], 'OM': [5, 7], 'OC': [4, 6]},
 {'SM': [0, 3], 'SC': [1, 2], 'OM': [5, 7], 'OC': [4, 6]},
 {'SM': [0, 3], 'SC': [1, 2], 'OM': [5, 7], 'OC': [4, 6]}]

In [47]:
count_bpress.append(len(sort_bpress[0]))

In [48]:
count_bpress

[1, 1, 1, 1, 1]

In [51]:
sub_bpress.append(count_bpress)

In [52]:
sub_bpress

[[1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]

In [93]:
sub_ses_labels[0]

['Re', 'SC', 'SM', 'OM', 'OC', 'SC', 'OM', 'SM', 'OC', 'Re']

In [100]:
a[1]

['Re', 'SC', 'SM', 'OM', 'OC', 'SC', 'OM', 'SM', 'OC', 'Re']

In [ ]:
sub_ses_labels[0]

In [102]:
b.append(len(sort_bpress[0]))

12

In [105]:
for x in range(1,12):
    print(x)

1
2
3
4
5
6
7
8
9
10
11


In [38]:
lab_indic_cor

{'SM': [1, 6], 'SC': [0, 4], 'OM': [2, 5], 'OC': [3, 7]}

In [39]:
behav

,Index,blk_name,begin_scan_btn.rt,begin_scan_btn.started,self_btn.keys,self_btn.rt,self_btn.started,other_n_btn.keys,other_n_btn.rt,other_n_btn.started,other_j_btn1.keys,other_j_btn1.rt,other_j_btn1.started
0,1,Rest_title.started,-1,-1.000000,-1,-1,-1.000000,-1,-1,-1.000000,-1,-1,-1.000000
1,5,MC_self_title.started,[1.4852273670258],420.648345,"['1', '1', '1', '1', '1', '1', '1', '1', '1', ...","[20.37084077997133, 45.95158576697577, 71.5679...",422.143973,-1,-1,-1.000000,-1,-1,-1.000000
2,7,MM_self_title.started,[1.4881828770739958],771.289100,"['1', '1', '1', '1', '1', '1', '1']","[51.46822209702805, 86.65644877194427, 125.071...",772.788834,-1,-1,-1.000000,-1,-1,-1.000000
3,9,MM_other_title.started,[1.4850854789838195],1123.567894,-1,-1,-1.000000,"['1', '1', '1', '1', '1', '1', '1']","[34.73545014404226, 39.43493033305276, 69.2674...",1125.501183,-1,-1,-1.000000
4,11,MC_other_title.started,[1.4840696200262755],1477.330291,-1,-1,-1.000000,"['1', '1', '1', '1', '1', '1', '1', '1', '1', ...","[12.868375902064145, 34.21847029705532, 57.018...",1479.062508,-1,-1,-1.000000
5,14,MC_self_title.started,[1.4890675380593166],1839.341232,"['1', '1', '1', '1', '1', '1', '1', '1', '1', ...","[26.03921395598445, 55.23933592799585, 82.4899...",1840.840920,-1,-1,-1.000000,-1,-1,-1.000000
6,16,MM_other_title.started,[1.484316918067634],2188.536311,-1,-1,-1.000000,-1,-1,-1.000000,"['1', '1', '1', '1', '1', '1', '1', '1', '1', ...","[86.63426328299101, 99.4346656659618, 110.7354...",2190.269583
7,18,MM_self_title.started,[1.4852906489977613],2535.515794,"['1', '1', '1', '1', '1', '1', '1', '1', '1', ...","[28.289827693952248, 47.468303015921265, 73.31...",2537.014617,-1,-1,-1.000000,-1,-1,-1.000000
8,20,MC_other_title.started,[1.4857432399876416],2880.727126,-1,-1,-1.000000,-1,-1,-1.000000,"['1', '1', '1', '1', '1', '1', '1', '1', '1', ...","[23.184997824020684, 46.602244504960254, 71.33...",2882.476853
9,23,Rest_title.started,-1,-1.000000,-1,-1,-1.000000,-1,-1,-1.000000,-1,-1,-1.000000


In [40]:
np.save(sav_work + sub + "bpress_rundata", sort_bpress, allow_pickle=True)


/jukebox/pkgs/PYGER/base/envs/0.11.0/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [ ]:
# Example visualization code -- BROKEN, will not work

In [ ]:
### Self - Run 2 ###
num_TR = 209
# Create a sequence of timepoints that a TR occurred on
time_points = list(range(1,num_TR+1)) #np.arange(0, (vdc_TRs_run - 1) * 1.5 + 1, 1.5)

# Plot the data
f, ax = plt.subplots(1,1, figsize = (12,5))
#ax.plot(time_points, stim_label_TR[0:vdc_TRs_run, 0], c='orange')
ax.plot(time_points, sc_press_2, c='orange')
ax.plot(sm_press_2, c='teal')

plt.legend(["Self Count", "Self Monitor"])
ax.set_ylabel('Button Press')
ax.set_xlabel('TRs')
ax.set_title("Self Run 2")